In [2]:
import os
import json
import gc
from collections import defaultdict
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import scipy
#765c79514570591907b84b82ebc2766e2c0a0f57
import networkx as nx

## Amirhossein Bagheri 98105621
## Mohammad Sadegh majidi Yazdi  98106004
## Amirmahdi kousheshi 98171053


## merging data

<b> skip 3 cell below if you want to use numpy object we already provided and ran this part and saved the output in Module_data you don't have to go throgh whole process it will takes you to downloads 3 Gb data and then needs 15 Gb ram to clean it and create final data.</b>
you have to download 4 topics data from [here](https://drive.google.com/drive/folders/19BL85jIEmxre3h8uj6BrFXAn4QoJz6zJ?usp=sharing) and put them in Data folder  it's going to look like this
![4 topics](./Module_data/pre_data.jpg)


after that run cell below
it will need 15 gb ram and will take about 5 minutes to run



In [3]:
# merge data
merged_data = {}
reduced_data = {}
for f in os.listdir("./DATA/"):
    f = os.path.join("./DATA",f)
    if f.endswith(".json"):
        print(f)
        data = json.load(open(f,"r"))
        for key in data:
            merged_data[key] = data[key]
            reduced_data[key] = {"referenceCount":data[key]["referenceCount"],"citationCount":data[key]["citationCount"],
            "authors":[l["authorId"] for l in data[key]["authors"]],
            "references":[{"pid" : l["paperId"], "authors":[k["authorId"] for k in l["authors"]]} for l in data[key]["references"]]}
        del data
        gc.collect()
json.dump(merged_data,open("./Module_data/data.json","w"))
json.dump(reduced_data,open("./Module_data/clean_data.json","w"))
print(len(merged_data))
del merged_data
del reduced_data
gc.collect()

./DATA/Language_Models.json
./DATA/transformers.json
./DATA/NLP.json
./DATA/natural_language_processing_transformers.json
29920


64

In [4]:
data = json.load(open("./Module_data/clean_data.json","r"))
i = 0
def count():
    global i
    j = i
    i += 1
    return j
x = 0
def count2():
    global x
    j = x
    x += 1
    return j
article_mapping = defaultdict(lambda : count())
author_mapping = defaultdict(lambda : count2())
for index_ar,key in enumerate(data):
    if key == None:
        continue
    article = data[key]
    article_mapping[key]
    for index,k in enumerate(article["authors"]):
        if k == None:
            continue
        author_mapping[k]
        if index > 5:
            break
    for index,k in enumerate(article["references"]):
        if k["pid"] == None:
            continue
        article_mapping[k["pid"]]
        for index2,m in enumerate(k["authors"]):
            if m == None:
                continue
            author_mapping[m]
            if index2 > 5:
                break
        if index > 9:
            break
    if index_ar > 1000:
        break

json.dump(article_mapping,open("./Module_data/article_mapping.json","w"))
json.dump(author_mapping,open("./Module_data/author_mapping.json","w"))
print(f"authors = {len(author_mapping)}")
print(f"articles = {len(article_mapping)}")
del data
del author_mapping
del article_mapping
gc.collect()

authors = 11449
articles = 6899


64

## create spare matrix 
here we create sparse matrix which is graph we need and store it 

In [5]:
data = json.load(open("./Module_data/clean_data.json","r"))
author_mapping = json.load(open("./Module_data/author_mapping.json","r"))
article_mapping = json.load(open("./Module_data/article_mapping.json","r"))
authors = np.zeros((len(author_mapping),len(author_mapping)))
articles = np.zeros((len(article_mapping),len(article_mapping)))
for index_ar,key in enumerate(data):
    if key == None:
        continue
    ar_id = article_mapping.get(key,None)
    if ar_id == None:
        continue
    article = data[key]
    authors_id_all = []
    for index,k in enumerate(article["authors"]):
        if k == None:
            continue
        aut_id = author_mapping.get(k,None)
        if aut_id == None:
            continue
        authors_id_all.append(aut_id)
        if index > 5:
            break
    for index,k in enumerate(article["references"]):
        if k["pid"] == None:
            continue
        ref_id = article_mapping.get(k["pid"],None)
        if ref_id == None:
            continue
        articles[ar_id][ref_id] += 1
        for index2,m in enumerate(k["authors"]):
            if m == None:
                continue
            ref_auth_if = author_mapping.get(m,None)
            if ref_auth_if == None:
                continue
            for auth_index in authors_id_all:
                authors[auth_index][ref_auth_if] += 1
            if index2 > 5:
                break
        if index > 9:
            break
    if index_ar > 1000:
        break
authors = csr_matrix(authors)
articles = csr_matrix(articles)
print(f"total number of authors = {len(author_mapping)}")
print(f"total number of edge in authors graph = {authors.sum()}")

print(f"total number of articles = {len(article_mapping)}")
print(f"total number of edge in authors graph = {articles.sum()}")

sparse.save_npz("./Module_data/authors_sparse.npz", authors)
sparse.save_npz("./Module_data/articles_sparse.npz", articles)
# your_matrix_back = sparse.load_npz("yourmatrix.npz")
del data
del author_mapping
del article_mapping
del articles,authors
gc.collect()

total number of authors = 11449
total number of edge in authors graph = 87202.0
total number of articles = 6899
total number of edge in authors graph = 9030.0


0